In [ ]:
# app.py
from flask import Flask, request, jsonify, render_template
import pandas as pd
import torch
from model import ANN  # Make sure ANN class is imported
import joblib  # For loading scaler and label encoder

# Initialize Flask
app = Flask(__name__)

# Load pretrained model
model = ANN()
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

# Load scaler and label encoder
scaler = joblib.load("scaler.pkl")
le = joblib.load("label_encoder.pkl")

# API endpoint to get prediction
@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json  # Get JSON payload
        df = pd.DataFrame([data])  # Convert to DataFrame
        # Preprocess
        X_scaled = scaler.transform(df)
        X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
        # Predict
        with torch.no_grad():
            outputs = model(X_tensor)
            _, predicted_idx = torch.max(outputs, 1)
        label = le.inverse_transform(predicted_idx.numpy())[0]
        return jsonify({"quality_label": label})
    except Exception as e:
        return jsonify({"error": str(e)})

# Route for simple frontend
@app.route('/')
def home():
    return render_template('index.html')  # HTML page

if __name__ == '__main__':
    app.run(debug=True)
